### This notebook performs spatial joins of the roads and VED coordinates to assign each point a speed limit (if possible) .

In [3]:
from shapely.geometry import Point, LineString
from shapely.wkt import loads
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt
import math
import os

Read the lines layer from OSM into a Pandas DataFrame

In [4]:
fields = ['WKT', 'osm_id',  'highway', 'maxspeed:forward', 'maxspeed:backward', 'source:maxspeed', 
          'maxspeed:advisory', 'maxspeed', 'maxspeed:practical']


dtype = {'maxspeed:forward': 'str', 'maxspeed:backward': 'str', 'source:maxspeed': 'str',
         'maxspeed:advisory': 'str', 'maxspeed': 'str', 'maxspeed:practical': 'str'}

roads_df = pd.read_csv('../data/osm/highways-lines-layer.csv', usecols=fields, dtype=dtype).fillna('0')
roads_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/osm/highways-lines-layer.csv'

Build a dictionary of roads, where keys are the OSM IDs and the values are the coordinates of the nodes in each road.

In [3]:
roads_arr = roads_df.to_numpy()
roads_dict = {roads_arr[i,1]: loads(roads_arr[i,0]) for i in range(len(roads_arr))}

Convert the roads DataFrame to GeoDataFrame.

In [4]:
roads_df['geometry'] = roads_df.WKT.apply(wkt.loads)
roads_df.drop('WKT', axis=1, inplace=True) #Drop WKT column

roads_gdf = gpd.GeoDataFrame(roads_df, geometry='geometry')
roads_gdf.head()

,osm_id,highway,maxspeed:forward,maxspeed:backward,source:maxspeed,maxspeed:advisory,maxspeed,maxspeed:practical,geometry
0,4411729,motorway_link,0,0,0,0,0,25 mph,"LINESTRING (-83.68891 42.30612, -83.68905 42.3..."
1,4439869,secondary,0,0,0,0,35 mph,0,"LINESTRING (-83.73924 42.23667, -83.73924 42.2..."
2,4569607,motorway_link,0,0,0,0,0,25 mph,"LINESTRING (-83.68558 42.30703, -83.68526 42.3..."
3,4569609,motorway_link,0,0,0,0,70 mph,0,"LINESTRING (-83.83737 42.28964, -83.83664 42.2..."
4,4574780,service,0,0,0,0,0,0,"LINESTRING (-83.77330 42.26921, -83.77309 42.2..."


Function that calculates the bearing of the two points whose coordinates are given in the parameters.

In [5]:
def get_bearing(lat1, lat2, lon1, lon2):
    """ Calculate the initial bearing for start point (lat1, lon1) and end point (lat2, lon2) """
    lat1, lat2 = math.radians(lat1), math.radians(lat2), 
    lon1, lon2 = math.radians(lon1), math.radians(lon2)
    dLon = lon2 - lon1
    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)
    brng = math.atan2(y, x)
    return brng

In [6]:
def mph2kph(speed):
    return round(speed * 1.609344)

def speed2numeric(speed):
    return mph2kph(int(speed.split()[0]))

In [7]:
def_maxspeed = {'residential': (1, speed2numeric('25 mph')), 
                'motorway': (1, speed2numeric('70 mph')), 
                'trunk': (1, speed2numeric('55 mph')),
                'primary': (1, speed2numeric('55 mph')),
                'secondary': (1, speed2numeric('45 mph')),
                'tertiary': (1, speed2numeric('35 mph')),
                'unclassified': (1, speed2numeric('55 mph')),
                'service': (1, speed2numeric('25 mph'))}


def get_speed(highway, maxspeed):
    """ Returns two values, the class of the speed limit and the speed limit."""
    if maxspeed['maxspeed'] != '0':
        return 0, speed2numeric(maxspeed['maxspeed'])
    elif maxspeed['maxspeed:forward'] != '0':
        sp = '{}-{}'.format(speed2numeric(maxspeed['maxspeed:forward']), 
                            speed2numeric(maxspeed['maxspeed:backward']))
        return -1, sp
    elif maxspeed['maxspeed:advisory'] != '0':
        return 2, speed2numeric(maxspeed['maxspeed:advisory'])
    elif maxspeed['maxspeed:practical'] != '0':
        return 3, str(speed2numeric(maxspeed['maxspeed:practical']))
    else:
        return def_maxspeed.get(highway, (np.nan, np.nan))

The function below looks a bit messy and could probably be written in a clearer and better way, but it does the job. It first checks if the current point has two different speed limits depending on the direction (is of class -1). If so, it saves the coordinates of this point and goes forward until it finds a point in the same trip that has different coordinates. It then uses these two points to calculate the bearing and decides on the direction of travel. The second case is when the road the point lies has the same speed limit regardless of the direction. In such case, it checks if the point that follows along the same trip is of class -1. If this is the case, it uses these two points to decide on the direction and updates all duplicate points (if any) that follow. 

In [8]:
# 1: VehId, 2: Trip, 4: Latitude, 5: Longitude, 22: matched_lat, 23: matched_lon, 24: match_type

def set_direction_sp(records):
    np_arr = np.zeros(len(records)) # set the size to the correct one
    for i in range(len(records)):
        p_lat1, p_lon1 = records[i, 22], records[i, 23]

        if records[i, 25] == -1 and np_arr[i] == 0:
            j = i
            osm_id = records[i, 27]
            road_lon1, road_lat1 = roads_dict[osm_id].coords[0]
            road_lon2, road_lat2 = roads_dict[osm_id].coords[-1]

            while (records[j, 4] == records[i, 4]) and \
                (records[j, 5] == records[i, 5]) and \
                (records[j, 1] == records[i, 1]) and \
                (records[j, 2] == records[i, 2]) and \
                (j < len(records)-1):
                j += 1

            if i != j:
                p_lat2, p_lon2 = records[j, 22], records[j, 23]
                # perform the computations and update all relevant rows
                br1 = get_bearing(p_lat1, p_lat2, p_lon1, p_lon2)
                br2 = get_bearing(road_lat1, road_lat2, road_lon1, road_lon2)
                direction = math.cos(br1-br2)
                speed = None

                while i <= j:
                    if records[i, 25] == -1:
                        if direction > 0: # same direction
                            # extract the correct speed limit and set it
                            speed = records[i, 26].split('-')[0]
                        else:
                            speed = records[i, 26].split('-')[1]
                    else:
                        speed = records[i, 26]
                    np_arr[i] = speed
                    i += 1
            else:
                np_arr[i] = records[i, 26]
        else:
            if np_arr[i] == 0:
                np_arr[i] = records[i, 26]
                j = i + 1
                if j < len(records):
                    if records[j, 25] == -1:
                        i = j
                        p_lat2, p_lon2 = records[i, 22], records[i, 23]
                        osm_id = records[i, 27]
                        road_lon1, road_lat1 = roads_dict[osm_id].coords[0]
                        road_lon2, road_lat2 = roads_dict[osm_id].coords[-1]
                        
                        while (records[j, 4] == records[i, 4]) and \
                        (records[j, 5] == records[i, 5]) and \
                        (records[j, 1] == records[i, 1]) and \
                        (records[j, 2] == records[i, 2]) and \
                        (j < len(records)-1):
                            j += 1
        
                        # perform the computations and update all relevant rows
                        br1 = get_bearing(p_lat1, p_lat2, p_lon1, p_lon2)
                        br2 = get_bearing(road_lat1, road_lat2, road_lon1, road_lon2)
                        direction = math.cos(br1-br2)
                        speed = None
                        while i <= j:
                            if records[i, 25] == -1:
                                if direction > 0: # same direction
                                    # extract the correct speed limit and set it
                                    speed = records[i, 26].split('-')[0]
                                else:
                                    speed = records[i, 26].split('-')[1]
                            else:
                                speed = records[i, 26]
                            np_arr[i] = speed
                            i += 1
    return np_arr

Read all files that contain the matched coordinates.

In [9]:
data_path = "../data/ved-valhalla/"
files = [os.path.join(data_path, file) for file in tqdm(os.listdir(data_path)) if file.endswith(".csv")]
files.sort()

  0%|          | 0/54 [00:00<?, ?it/s]

Converting each into a Pandas GeoDataFrame, spatially joining of the two GeoDataFrames and removing the duplicates which are as a result of intersections.

In [10]:
def spatial_join(file):
    data_df = pd.read_csv(file)
    data_gdf = gpd.GeoDataFrame(data_df, geometry=gpd.points_from_xy(data_df.matched_lon, data_df.matched_lat))
    sjoined_gdf = gpd.sjoin_nearest(data_gdf, roads_gdf, how="left", max_distance=1e-05, distance_col='distance')
    sjoined_gdf_grouped = sjoined_gdf.groupby(sjoined_gdf.index).first() # remove duplicates
    gdf_cols = ['osm_id', 'highway', 'maxspeed:forward', 'maxspeed:backward', 
                'maxspeed:advisory', 'maxspeed', 'maxspeed:practical']
    data_df = data_df.drop('geometry', 1)
    data_df = data_df.merge(sjoined_gdf_grouped[gdf_cols], left_index=True, right_index=True)
    return data_df

def add_speed_limits(file):
    data_df = spatial_join(file)
    columns = [col for col in data_df.columns]
    maxspeed_cols = list(range(columns.index('highway'), len(columns)))

    data_arr = data_df.to_numpy()
    N = len(data_arr)
    ved_out_df = np.c_[data_arr, np.zeros(N), np.empty(N, dtype='<U10')]
    for row in ved_out_df:
        # ms_cols dictionary: keys: maxspeed tag, values: speed limit
        ms_cols = {columns[j]: row[j] for j in range(27, len(columns))}
        if row[26] is None:
            sp_cls, speed_limit = np.nan, np.nan
        else:
            sp_cls, speed_limit = get_speed(row[26], ms_cols)
        row[32], row[33] = sp_cls, speed_limit

    columns.extend(['Class of Speed Limit', 'Speed Limit[km/h]'])
    ved_out_df = pd.DataFrame(ved_out_df, columns=columns) 
    ved_out_df = ved_out_df.drop([columns[i] for i in maxspeed_cols], axis=1)
    osm_col = ved_out_df['osm_id']
    ved_out_df = ved_out_df.drop(['osm_id'], 1)
    ved_out_df = ved_out_df.merge(osm_col, left_index=True, right_index=True)
    speed_column = set_direction_sp(ved_out_df.to_numpy())
    speed_column = pd.DataFrame(speed_column, columns={'Speed Limit with Direction'})
    ved_out_df = ved_out_df.merge(speed_column, left_index=True, right_index=True)
    return ved_out_df

In [ ]:
def get_save_path(filename):
    return f"../data/ved-speed-limits/{filename[len(data_path)-1:]}"

for file in tqdm(files):
    ved_file = add_speed_limits(file)
    ved_file = ved_file.drop('osm_id', 1)
    ved_file.to_csv(get_save_path(file), na_rep=np.nan, index=False)
    ved_file

  0%|          | 0/54 [00:00<?, ?it/s]